# 순환 신경망 구현 및 학습

In [1]:
import tensorflow as tf

#하이퍼 파라미터

In [2]:
EPOCHS = 10
NUM_WORDS = 10000 #10000개의 단어만 사용하겠습니다.

#모델 정의  
embedding : 특정단어들이 원 핫벡터에 하나씩 배정되는데, 특정 단어를 원한벡터로 하면 잘 안된다. 정수값이기 때문이다. 그래서 우리가 사용하는 실수로 가져오기위해 길이가 10000 원핫 벡터를 길이가 16인 feature vercor로 바꿔주는게 embedding의 역할입니다.

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.rnn = tf.keras.layers.LSTM(32)
        self.dense = tf.keras.layers.Dense(2, activation='softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

#학습, 테스트 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

#데이터 셋 준비

In [5]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=32)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                      value=0,
                                                      padding='pre',
                                                      maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

#학습 환경 정의
## 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [6]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

#학습 루프 동작

In [7]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.5009668469429016, Accuracy: 74.66799926757812, Test Loss: 0.43555381894111633, Test Accuracy: 79.41600036621094
Epoch 2, Loss: 0.3696795105934143, Accuracy: 83.51200103759766, Test Loss: 0.44655802845954895, Test Accuracy: 79.00399780273438
Epoch 3, Loss: 0.31699296832084656, Accuracy: 86.29199981689453, Test Loss: 0.4646114408969879, Test Accuracy: 78.56800079345703
Epoch 4, Loss: 0.2635522186756134, Accuracy: 88.96800231933594, Test Loss: 0.5097373127937317, Test Accuracy: 77.91600036621094
Epoch 5, Loss: 0.20587608218193054, Accuracy: 91.63999938964844, Test Loss: 0.6313369870185852, Test Accuracy: 76.34400177001953
Epoch 6, Loss: 0.16270530223846436, Accuracy: 93.46800231933594, Test Loss: 0.7402276396751404, Test Accuracy: 76.00399780273438
Epoch 7, Loss: 0.12017648667097092, Accuracy: 95.43600463867188, Test Loss: 0.9673022627830505, Test Accuracy: 75.53600311279297
Epoch 8, Loss: 0.09436867386102676, Accuracy: 96.4800033569336, Test Loss: 1.093193769454956, Test